need to calculate how to include shorts in positions
how to get risk free rate to calculate sharpe
calculate a rolling beta to an asset y
calculate a daily VAR if possible

In [1]:
from ipynb.fs.full.quick_functions import *
import math
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import plot
import pandas as pd
import numpy as np
from scipy.stats import norm

df = full_df(10)
df = delete_outliers(df)
df_indexed = df / df.iloc[0]

[*********************100%%**********************]  25 of 25 completed
[*********************100%%**********************]  25 of 25 completed


In [2]:
def create_monthly_heatmap(data):
    month_dict = {1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun',
                  7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'}
    monthly_returns = df_indexed["portfolio"].resample('M').last().pct_change() * 100
    heatmap_data = monthly_returns.groupby([monthly_returns.index.year, monthly_returns.index.month]).last().unstack()
    heatmap_data.columns = [month_dict[month] for month in heatmap_data.columns]
    fig = go.Figure(data=go.Heatmap(
        z=heatmap_data.values,
        x=heatmap_data.columns,
        y=heatmap_data.index,
        colorscale='RdYlGn',
        zmin=-5,  # You can adjust these for your scale
        zmax=5,
        showscale=True
        ))
    fig.update_layout(
        title='Monthly Returns Heatmap',
        xaxis_nticks=12,
        yaxis=dict(type='category', autorange='reversed'),  # Reversing the y-axis
        height=400, width=800
    )

    #fig.show()
    return(fig)
    
def create_daily_return_histogram(series):
    daily_returns = series.pct_change() * 100
    fig = go.Figure(data=[go.Histogram(x=daily_returns, nbinsx=100, histnorm='percent')])  # Add histnorm='percent' here
    fig.update_layout(
        title='Sample Portfolio Daily Returns',
        xaxis_title='Daily Returns',
        yaxis_title='% of Observations',
        bargap=0.05,
        height=400,
        width=800# gap between bars of adjacent location coordinates
    )
    #fig.show()
    return(fig)
    
def create_perf_charts(df,lookback,positions,direction=None):
    dfc = df.iloc[-lookback:].copy()
    df_indexed = dfc / dfc.iloc[0]
    length = len(positions)
    initial_weight = 100 / length
    weights = [initial_weight for x in range(length)]
    df_indexed["portfolio"] = df_indexed[positions].mul(weights, axis=1).sum(axis=1)
    length_days = len(dfc.index)
    portfolio = df_indexed["portfolio"]
    total_return = portfolio.iloc[-1] / portfolio.iloc[1]
    annual_return = (total_return ** (252 / length_days) - 1) * 100
    daily_standard_deviation = portfolio.pct_change(1).std()
    annual_standard_deviation = daily_standard_deviation * math.sqrt(252) * 100
    return_to_deviation = annual_return / annual_standard_deviation
    drawdowns_1m = (portfolio / portfolio.rolling(20).max() - 1) * 100
    daily_changes = portfolio.pct_change(1) * 100
    yoy_returns = portfolio.pct_change(252) * 100

    title = f"Annual Return {round(annual_return,2)} st. dev {round(annual_standard_deviation,2)},return_to_deviation {round(return_to_deviation,2)}"
    fig = make_subplots(rows=4, cols=1, row_heights=[2,1,1,1])
    fig.add_trace(go.Scatter(x=df_indexed.index, y=df_indexed["portfolio"], mode='lines', name='Portfolio'), row=1, col=1)
    fig.add_trace(go.Scatter(x=df_indexed.index, y=yoy_returns, mode='lines', name='YoY Returns'), row=2, col=1)
    fig.add_trace(go.Scatter(x=dfc.index, y=drawdowns_1m, name='1m drawdowns', mode='lines'), row=3, col=1)
    fig.add_trace(go.Bar(x=dfc.index, y=daily_changes,  name='Daily Returns', marker_color='black'), row=4, col=1)
    fig.update_layout(height=600, width=1000, title_text=title, showlegend=True, plot_bgcolor='white')
    #fig.show()
    return(fig,df_indexed)
    
def create_ten_worst(df_indexed):
    df_indexed["daily_returns"] = df_indexed["portfolio"].pct_change()
    worst_ten = df_indexed.sort_values(by="daily_returns").head(10).index.tolist()
    worst_ten_table = round(df_indexed.loc[worst_ten]["daily_returns"] * 100,2)
    dfx = pd.DataFrame(worst_ten_table)
    dfx.columns = ["Worst Ten Days"]
    return(dfx)

def create_correlation_grid(df):
    correlation_matrix = df.pct_change(1)[-200:].corr().round(2)
    mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
    masked_corr = correlation_matrix.mask(mask)
    fig = go.Figure(data=go.Heatmap(
        z=masked_corr,
        x=correlation_matrix.columns,
        y=correlation_matrix.index,
        colorscale='RdYlGn'))

    fig.update_layout(
        title='Correlation Matrix',
        xaxis_title='Asset',
        yaxis_title='Asset',
        xaxis=dict(tickangle=-45),
        yaxis=dict(tickmode='array', tickvals=np.arange(len(correlation_matrix.columns))),
        height=400, width=800
    )
    return(fig)


def calculate_var(df):
    returns = df.pct_change().dropna()
    length = len(df.columns)
    initial_weight = 100 / length
    weights = pd.DataFrame([initial_weight for x in range(length)])
    mean_return = returns.mean().dot(weights)
    cov_matrix = returns.cov()
    portfolio_std = np.sqrt(weights.T.dot(cov_matrix).dot(weights))
    confidence_level = 0.95
    portfolio_var = norm.ppf(1 - confidence_level, mean_return, portfolio_std)
    print(f'Portfolio VaR (95% confidence): {portfolio_var * 100:.2f}%')


In [5]:
positions = ["BRL","MXN","COP","HUF","JPY"]
years_lookback = 3
lookback = 252 * years_lookback

#the df here is a dataframe of positions as columns, index of dates and values of total return indices

fig1, df_indexed = create_perf_charts(df,lookback,positions)
fig2 = create_monthly_heatmap(df_indexed["portfolio"])
fig3 = create_daily_return_histogram(df_indexed["portfolio"]) 
fig4 = create_correlation_grid(df_indexed[positions])

file_path = 'portfolio_tearsheet.html'
if os.path.exists(file_path):
    os.remove(file_path)
    
with open('portfolio_tearsheet.html', 'a') as f:
    for figure in [fig1, fig2, fig3, fig4]:
        f.write(figure.to_html(full_html=False, include_plotlyjs='cdn'))
    

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
fig = create_correlation_grid(df_indexed[positions])
fig.show()

In [7]:
df = get_fred("DGS10",10)
df["cumulative"] = df.diff(1).cumsum() * 100
fig = px.line(df["cumulative"])
fig.show()
    
    

<IPython.core.display.Javascript object>